In [1]:
##### CHANGE THIS TO YOUR FOLDER##################
##################################################
bigfol=r"C:\Users\Image_Processing_PC\OneDrive\OneDrive - UNSW\Image_Analysis\NewMicroscope\Time_series"
##################################################
#################################################






#initialise dependencies


from matplotlib import pyplot as plt
from numpy.linalg import norm
import numpy as np


from scipy import sparse
from imutils.object_detection import non_max_suppression


from pathlib import Path
from skimage.color import rgb2lab, lab2rgb
from skimage.feature import match_template
from skimage.feature import peak_local_max

from os import listdir
import os
import glob
from PIL import Image
import rawpy
import imageio


from PIL import Image
import PIL

from PIL.ExifTags import TAGS
import imutils

import os
#import hcluster

import re
from datetime import datetime
import pandas as pd
import time

from math import pow
import scipy.signal 
%matplotlib qt
#template matching

import cv2 as cv
import numpy as np

import os
import sys
from tqdm.notebook import tqdm
from time import sleep
from skimage.filters import threshold_otsu, threshold_local
from skimage.filters import threshold_otsu, rank
from skimage.util import img_as_ubyte





#General Functions
def listdirs(folder):
## a function which lists the files in a folder and adds to a list. returns a list of folders, 
##input folder: the file path to folder
    return [
        d for d in (os.path.join(folder, d1) for d1 in os.listdir(folder))
        if os.path.isdir(d)
    ]

def makeclippingmask(image):
    

    #makes a clipping mask around each bright spot so the analysis isn't thrown off
    grayA = cv.cvtColor(image, cv.COLOR_RGB2GRAY)
    blurred = cv.GaussianBlur(grayA, (11,11), 0)

    #watershed thresholding. Based on: https://docs.opencv.org/3.4/d2/dbd/tutorial_distance_transform.html
    src = image.copy()
    
    
    # Create a kernel that we will use to sharpen our image
    # an approximation of second derivative, a quite strong kernel
    kernel = np.array([[1, 1, 1], [1, -8, 1], [1, 1, 1]], dtype=np.float32)
    # do the laplacian filtering as it is
    # well, we need to convert everything in something more deeper then CV_8U
    # because the kernel has some negative values,
    # and we can expect in general to have a Laplacian image with negative values
    # BUT a 8bits unsigned int (the one we are working with) can contain values from 0 to 255
    # so the possible negative number will be truncated
    imgLaplacian = cv.filter2D(src, cv.CV_32F, kernel)
    sharp = np.float32(src)
    imgResult = sharp - imgLaplacian
    # convert back to 8bits gray scale
    imgResult = np.clip(imgResult, 0, 255)
    imgResult = imgResult.astype('uint8')
    imgLaplacian = np.clip(imgLaplacian, 0, 255)
    imgLaplacian = np.uint8(imgLaplacian)
    #cv.imshow('Laplace Filtered Image', imgLaplacian)
    #cv.imshow('New Sharped Image', imgResult)
    
    # Create binary image from source image
    # Create binary image from source image
    bw = cv.cvtColor(imgResult, cv.COLOR_BGR2GRAY)
   # _, bw2 = cv.threshold(grayA, 30, 255, cv.THRESH_BINARY | cv.THRESH_OTSU)
    #plt.imshow(bw)
    bw = cv.adaptiveThreshold(bw, 255, cv.ADAPTIVE_THRESH_MEAN_C, cv.THRESH_BINARY, 37, 1)
    #cv.imshow('Binary Image', bw)
    
    
    opening = cv.morphologyEx(bw,cv.MORPH_OPEN,kernel, iterations = 3)
    
    
    
    # sure background area
    sure_bg = cv.dilate(opening,kernel,iterations=3)
    
    # Finding sure foreground area
    dist_transform = cv.distanceTransform(opening,cv.DIST_L2,5)
    # Threshold to obtain the peaks
    # This will be the markers for the foreground objects
    _, sure_fg = cv.threshold(dist_transform, 0.2, 1.0, cv.THRESH_BINARY)
    # Dilate a bit the dist image
    kernel1 = np.ones((3,3), dtype=np.uint8)
    sure_fg = cv.dilate(sure_fg, kernel1)
    ret, markers = cv.connectedComponents(np.uint8(sure_fg))
    
    
    #cv.imshow('Final Result', sure_fg)
    # Finding unknown region
    sure_fg = np.float32(sure_fg)
    sure_bg = np.float32(sure_bg)
    unknown = cv.subtract(sure_bg,sure_fg)
    
    #cv.imshow('Distance Transform Image', dist_transform)
    
    # Marker labelling
    

    # Add one to all labels so that sure background is not 0, but 1
    markers = markers+1

    # Now, mark the region of unknown with zero
    markers[unknown==255] = 0
    #print(markers.shape)
    
    markers = cv.watershed(imgResult,markers)
    #print(markers)
    
    mask2 = np.zeros(image.shape[:2], dtype= np.uint8)
    mask2[markers >1] = [255]
    
    #colours=((255,255,255))
    # Fill labeled objects with random colors
    #for i in range(markers.shape[0]):
    #    for j in range(markers.shape[1]):
    #        index = markers[i,j]
    #        if index>0:
    #            mask2[i,j] = 255
    

    image_rgb=image.copy()
    image_blocked = cv.bitwise_and(image_rgb, image_rgb, mask=mask2)
    #plt.imshow(image_blocked)
    return image_blocked

def imgregfun(imagebef, imageafter):
#### A function for image registration, stolen of the internet but I can't remember where from
###Inputs: imagebef- the before image
##########imageafter- the after image
###outputs: transimaf- the translated after image
    # Open the image files.
    img1_color = imageafter  # Image to be aligned.
    img2_color = imagebef  # Reference image.

    # Convert to grayscale.
    img1 = cv.cvtColor(img1_color, cv.COLOR_BGR2GRAY)
    img2 = cv.cvtColor(img2_color, cv.COLOR_BGR2GRAY)
    height, width = img2.shape

    # Create ORB detector with 5000 features.
    orb_detector = cv.ORB_create(5000)

    # Find keypoints and descriptors.
    # The first arg is the image, second arg is the mask
    #  (which is not reqiured in this case).
    kp1, d1 = orb_detector.detectAndCompute(img1, None)
    kp2, d2 = orb_detector.detectAndCompute(img2, None)

    # Match features between the two images.
    # We create a Brute Force matcher with
    # Hamming distance as measurement mode.
    matcher = cv.BFMatcher(cv.NORM_HAMMING, crossCheck=True)

    # Match the two sets of descriptors.
    matches = matcher.match(d1, d2)

    # Sort matches on the basis of their Hamming distance.
    matches.sort(key=lambda x: x.distance)

    # Take the top 90 % matches forward.
    matches = matches[:np.int(len(matches) * 90)]
    no_of_matches = len(matches)

    # Define empty matrices of shape no_of_matches * 2.
    p1 = np.zeros((no_of_matches, 2))
    p2 = np.zeros((no_of_matches, 2))

    for i in range(len(matches)):
        p1[i, :] = kp1[matches[i].queryIdx].pt
        p2[i, :] = kp2[matches[i].trainIdx].pt

    # Find the homography matrix.
    homography, mask = cv.findHomography(p1, p2, cv.RANSAC)

    # Use this matrix to transform the
    # colored image wrt the reference image.
    transformed_img = cv.warpPerspective(img1_color, homography,
                                          (width, height))
    transimaf=transformed_img
    return transimaf
def imageprocessingfunction(beforefol,afterfol):
###A function for getting all the CR2 files within the before and after folders, then reading them
### and saving them on the disk as virtual images 
##########################################################################
###Inputs: beforefol: selected before folder
##########afterfol: selected after folder 
###Outputs: imaf: the images in the after folder as an array
###########imbef: the images in the before folder as an array 
###########beforeimfile: the list of before image files
###########afterimfile: the list of after image files
    # Get file list
    beforeimfile=glob.glob(beforefol+"\\"+"*.CR2")
    afterimfile=glob.glob(afterfol+"\\"+"*.CR2")
    #print(afterimfile)

    #Exifdata is just there in case you need to edit the images in a fancy way.
    imaf,labaf,imbef,labef=[],[],[],[]
    for impath in afterimfile:
        image,exifdata=   convertfilefun(impath)
        imaf.append(np.dstack((image)))
        labaf.append(exifdata)
    for impath in beforeimfile:
        image,exifdata= convertfilefun(impath)
        imbef.append(np.dstack((image)))
        labef.append(exifdata)
    return imaf,imbef,beforeimfile,afterimfile

def convertfilefun(path):
## a function which converts CR2 images to TIFF images the computer can actually read
## input: path- path to raw image
## output : an image that is readable using cv2
    with rawpy.imread(path) as raw:
        #Can fiddle with camera settings but I wouldn't reccoment it
        rgb = raw.postprocess(use_camera_wb=True,
                              no_auto_bright=True,
                              gamma=(2.222, 4.5),
                              chromatic_aberration=(1, 1))
        #cv2.imwrite(path + '.tiff',rgb)
        # extract EXIF data to save as metadata
        metdat = Image.open(path)
        exifdata = metdat.getexif()
        image = rgb
        image = rgb.reshape(
            (1, image.shape[0], image.shape[1], image.shape[2]))
        return image, exifdata
        #plt.imsave(path + '.png',rgb)
        #g=print(path + '.png')
        #return g
        
def saveexcelfun(dimercoord,corecoord,savepath):
#Save the coordinates of everything to an excel spreadsheet. Yeah I know it's ugly. 
#input: dimercoord- the dimer coordinates
########corecoord- the core coordinates 
########savepath- the folder location where the files will be saved 
    columns=['Before Dimers xval']
    saveexcel=savepath+"."+"xlsx"
    beforedimercentres = pd.DataFrame({'Before Dimers xval':dimercoord})
    try:
        aftercorecentres = pd.DataFrame({'After core xval': corecoord})
    except:
        aftercorecentres=pd.DataFrame({'After core xval': np.array([0]), 
                                'After core yval': np.array([0])})

   
    writer = pd.ExcelWriter(saveexcel,engine='xlsxwriter')
    workbook=writer.book
    worksheet=workbook.add_worksheet('DimersPicked')
    writer.sheets['DimersPicked'] = worksheet
    worksheet2=workbook.add_worksheet('CoresPicked')
    writer.sheets['CoresPicked'] = worksheet2


    beforedimercentres.to_excel(writer,sheet_name='DimersPicked',startrow=1 , startcol=0)
    #worksheet.write_string(beforedimercentres.shape[0] + 4, 0, beforedimercentres.name)

    aftercorecentres.to_excel(writer,sheet_name='CoresPicked',startrow=1, startcol=3)
    

    writer.save()
def savetextfilefun(data,savepath,datastring):
##### A function which saves an array to a text file. Is a little buggy in that sometimes there's weird spaces. 
##### reccomend the excel save functions instead. Python struggles to re-read these text tiles

    savetextstring=savepath+datastring+".txt"
    file = open(savetextstring,"w")
    for dataentry in data:
        arr_of_strings = np.array2string(dataentry)
        file.write(arr_of_strings) 
    file.close() 
    

def load_images_from_folder(folder):
## A function which reads in images and adds them to a list of images.
## input: folder- a string which points to the folder location
## output: images, a list of images within the folder
## note !! The images wil be read in with open cv, and will be in BGR format and will look strange unless converted
    images = []
    for filename in os.listdir(folder):
        img = cv.imread(os.path.join(folder,filename))        
        if img is not None:
            images.append(img)
    return images
def load_images_from_foldercv(folder):
## A function which reads in images and adds them to a list of images. This function also converts them to RGB
## input: folder- a string which points to the folder location
## output: images, a list of images within the folder

    images = []
    for filename in os.listdir(folder):
        img = cv.imread(os.path.join(folder,filename))
       
        if img is not None:
            img= cv.cvtColor(img, cv.COLOR_BGR2RGB)
            images.append(img)
            
    return images
def load_images_from_folderhsv(folder):
## A function which reads in images and adds them to a list of images. This function also converts them to HSV
## input: folder- a string which points to the folder location
## output: images, a list of images within the folder
    images = []
    for filename in os.listdir(folder):
        img = cv.imread(os.path.join(folder,filename))
        img= cv.cvtColor(img,  cv.COLOR_BGR2HSV)
        if img is not None:
            images.append(img)
    return images

#####################################################
###################################################
#Template matching functions


def findidealimagescale(image,template):
    # loop over the images to find the template in
   
        # load the image, convert it to grayscale, and initialize the
        # bookkeeping variable to keep track of the matched region
        
    gray = cv.cvtColor(image, cv.COLOR_RGB2GRAY)
    template=cv.cvtColor(template, cv.COLOR_HSV2RGB)
    template=cv.cvtColor(template, cv.COLOR_RGB2GRAY)
    #template.astype(np.uint8)
    #gray.astype(np.uint8)
    found = None
    scalefin= None
    (h, w) = template.shape[:2]
    i=0
    # loop over the scales of the image
    for scale in np.linspace(0.2, 1.0, 20)[::-1]:
        # resize the image according to the scale, and keep track
        # of the ratio of the resizing
        resized = imutils.resize(gray, width = int(gray.shape[1] * scale))
        r = gray.shape[1] / float(resized.shape[1])
        # if the resized image is smaller than the template, then break
        # from the loop
        if resized.shape[0] < h or resized.shape[1] < w:
            break
        result = cv.matchTemplate(gray, template, cv.TM_SQDIFF_NORMED)
        (minval, _, minloc, _) = cv.minMaxLoc(result)
        if found is None or minval < found:
            found = minval
            scalefin=scale
    return scalefin 
        
# Malisiewicz et al.
def non_max_suppression_fast(boxes, overlapThresh):
    # if there are no boxes, return an empty list
    if len(boxes) == 0:
        return []
    # if the bounding boxes integers, convert them to floats --
    # this is important since we'll be doing a bunch of divisions
    if boxes.dtype.kind == "i":
        boxes = boxes.astype("float")
    # initialize the list of picked indexes	
    pick = []
    # grab the coordinates of the bounding boxes
    x1 = boxes[:,0]
    y1 = boxes[:,1]
    x2 = boxes[:,2]
    y2 = boxes[:,3]
    # compute the area of the bounding boxes and sort the bounding
    # boxes by the bottom-right y-coordinate of the bounding box
    area = (x2 - x1 + 1) * (y2 - y1 + 1)
    idxs = np.argsort(y2)
    # keep looping while some indexes still remain in the indexes
    # list
    while len(idxs) > 0:
        # grab the last index in the indexes list and add the
        # index value to the list of picked indexes
        last = len(idxs) - 1
        i = idxs[last]
        pick.append(i)
        # find the largest (x, y) coordinates for the start of
        # the bounding box and the smallest (x, y) coordinates
        # for the end of the bounding box
        xx1 = np.maximum(x1[i], x1[idxs[:last]])
        yy1 = np.maximum(y1[i], y1[idxs[:last]])
        xx2 = np.minimum(x2[i], x2[idxs[:last]])
        yy2 = np.minimum(y2[i], y2[idxs[:last]])
        # compute the width and height of the bounding box
        w = np.maximum(0, xx2 - xx1 + 1)
        h = np.maximum(0, yy2 - yy1 + 1)
        # compute the ratio of overlap
        overlap = (w * h) / area[idxs[:last]]
        # delete all indexes from the index list that have
        idxs = np.delete(idxs, np.concatenate(([last],
            np.where(overlap > overlapThresh)[0])))
    # return only the bounding boxes that were picked using the
    # integer data type
    return boxes[pick].astype("int")
        
def nonmaxsuppression(xCoords,yCoords,template):
    center_coordinates=[]
    rects=[]
    rectangle_coordinates=[]
    (w, h) = template.shape[:2]
    #print(w)
    #print(h)
## stops the overcounting of variables with nonmax suppression and returns an updated list
    for (x, y) in zip(xCoords, yCoords):
    # update our list of rectangles
        rects.append((x, y, x +w, y + h))
    picked_rectangles=non_max_suppression_fast(np.array(rects),0.5)
        #I hate how opencv does rectangles, so arrange these to finds the centres
    for (startX, startY, endX, endY) in picked_rectangles:
        #the coordinates are top left hand corner, bottom right hand corner for rectangles 
        center_coordinates.append((startX+h//2,startY+h//2))
        rectangle_coordinates.append((startX, startY, endX, endY))
     
    #print("center coordinates are ",center_coordinates)
    #print("rectangle_coordinates are ", rectangle_coordinates)
    return center_coordinates,rectangle_coordinates,w,h

def templatematchingalgorithm(img_hsv,template, method,threshold):
# defines the template matching algorithm and finds the minimum locations
#inputs: img_rgb- the image to be matched in HSV format
#########template- the template to be matched in HSV format
#########method- the method of determining the minim. cv.TM_SQDIFF_NORMED is good for colour
##### extras found here: https://docs.opencv.org/master/df/dfb/group__imgproc__object.html#ga3a7850640f1fe1f58fe91a2d7583695d
#########threshold: the threshold where the minimum is defined. Variable. May want to do something with min_val and max
#outputs: locations_of_minimum- a really big array that needs to be zipped. 
#Note: this works on colour (3 d) images but may want to change to just hue
    img_hsv = cv.cvtColor(img_hsv, cv.COLOR_HSV2BGR)
    img_hsv = cv.cvtColor(img_hsv, cv.COLOR_BGR2Lab)
    _,img_hsv_a,img_hsv_b=cv.split(img_hsv)
    template = cv.cvtColor(template, cv.COLOR_HSV2BGR)
    template = cv.cvtColor(template, cv.COLOR_BGR2Lab)
    _,template_a,template_b=cv.split(template)
    
    mat_of_matching_results_a=cv.matchTemplate(img_hsv_a,template_a,method)
    mat_of_matching_results_b=cv.matchTemplate(img_hsv_b,template_b,method)
    
    #This is to get some details about the minimum but isn't actually used
    #print("template matching done")
    (min_val_a, max_val_a, _, max_loc_a) = cv.minMaxLoc(mat_of_matching_results_a)
    (min_val_b, max_val_b, _, max_loc_b) = cv.minMaxLoc(mat_of_matching_results_b)
    
    #print("The min is done")
    #print(min_val)
    #print(max_val_a)
    if max_val_a>threshold:
        #threshold=0.6
        (yCoords, xCoords) = np.where(np.logical_and(mat_of_matching_results_a >= threshold, mat_of_matching_results_b>=threshold))
        while (len(xCoords))>2500000:
            threshold=threshold+0.1
            #print(threshold)
            (yCoords, xCoords) = np.where(np.logical_and(mat_of_matching_results_a >= threshold, mat_of_matching_results_b>=threshold))
            
    
        center_coordinates,rectangle_coordinates,w,h=nonmaxsuppression(xCoords,yCoords,template_a)
    else:
        center_coordinates=[[0,0]]
        rectangle_coordinates=[[0,0,0,0]]
        w=0.00001
        h=0.00001
    
    #print(locations_of_minimum)
    return center_coordinates,rectangle_coordinates,w,h



    
    
def imagetempmatch(imgpath,img_rgb,threshold,clumpflag):
## performs the template matching function on each template image found in imgpath
## input: imgpath: a string pointing to the folder the template images are contained in
##########img_rgb: the image to be matched, in rgb format 
##########threshold: the threshold at which the minimum is accepted. The minimum value is the value where the 
#####################template matching function thinks that the image is matched. Note, this may be a maximum 
####################for other methods
    #loads the template images in as HSV
    images=load_images_from_folderhsv(imgpath)
    center_list=[]
    rectangle_list=[]
    r_list=[]
    #Creates a mask which has the shape of the image to be matched. The dtype is important or error will occur. 
    # This mask is to test whether the template matching has counted the same point multiple times
    mask = np.zeros(img_rgb.shape, dtype=np.uint8)
    res=[];
    if clumpflag==1:
        scale=findidealimagescale(img_rgb,images[0])
    else:
        scale=1
    #print(scale)
    resized = imutils.resize(img_rgb, width = int(img_rgb.shape[1]*scale))
    for template in tqdm(images):
        #for each template, the width and height is taken
        if clumpflag==1:
            scale=findidealimagescale(img_rgb,template)
        else:
            scale=1
        #print(scale)
        resized = imutils.resize(img_rgb, width = int(img_rgb.shape[1]*scale))

        w=16
        h=16
        
        
        #big image converted to hsv format
        img_hsv= cv.cvtColor(resized,  cv.COLOR_RGB2HSV)
        #the location minima identified with the template matching algorithm
        center_coordinates,rectangle_coordinates,w,h=templatematchingalgorithm(img_hsv,template,cv.TM_CCOEFF_NORMED,threshold)
        center_list.extend(center_coordinates)
        rectangle_list.extend(rectangle_coordinates)
        r_list.append(w)
        
        #print(center_coordinates)
        
        #print("locations of min are", locations_of_min)
        # the locations are checked for multiple counting of the same point. 
        #particle_count=checkfordoublecounting(img_hsv,locations_of_min)
        #print(particle_count)
    #print(np.array(center_list))
    return np.array(center_list),np.array(rectangle_list),np.max(np.array(r_list)),h


def blockimagecombo(image_rgb, points,r,bok):
    #print(points)
    if bok==1:
        image_blocked=image_rgb.copy()
        for pt in points:
            
            image_blocked = cv.circle(image_blocked,pt,r, (255,255,255), -1)
    else:
        mask2 = np.zeros(image_rgb.shape[:2], dtype= np.uint8)
        for pt in points: 
            #print(pt)
            mask2 = cv.circle(mask2,pt,r, (255,255,255), -1)
            
            # a rectangle is drawn on the mask, which marks where the points are 
        #invmask=255-mask2
        #This is an inbuilt cv function which clips the image around the mask. 
        #plt.imshow(cv.bitwise_and(image_rgb, image_rgb, mask=invmask))
        image_blocked = cv.bitwise_and(image_rgb, image_rgb, mask=mask2)
    return image_blocked


def performtemplatematching(image_rgb,imgpath,bok,threshold,clumpflag):
## a function which performs template matching on the images and blocks them depending on whether
## we want to keep the particles or block them out for more accuracy 
#input: image_rgb: the big image in rgb format where we are looking for matches
########imgpath: the path to the template images, as a string
#########bok: block or keep. 0 is for keeping, 1 is for blocking with a rectange
#########threshold: the threshold for the minimum values. Variable. 
    
    center_coordinates,rectangle_coordinates,w,h=imagetempmatch(imgpath,image_rgb,threshold,clumpflag)
    r=round(w/1.5)

    correctedimg=blockimagecombo(image_rgb,center_coordinates ,r,bok)
    return correctedimg,center_coordinates,r

def performsaveimage(image,path):
## saves image using pillow, which is a lot faster than matplot lib. 
    img_rgb_corr=image
    try:
        im_pil = Image.fromarray(img_rgb_corr)
        im_pil.save(path, compress_level=1)
    except:
        exception=1

def savetotrainingfol(foldername,image,points):
## saves the cropped images to a folder, for use in machine learning. uses a 16 pixel box.
#input: foldername: name of the folder where you want the images stored
#######image: an image in rgb format which you want to cut up
########points: the coordinates of the particles which you have selected. 
    boxwid=round(16/2)
    w=16
    for j,pt in enumerate(points):
        savepathfol= addstringwithtime(foldername +'\\')
        savepath= savepathfol+str(j)+"registeredimg" + "." + "png"
        #savepath=os.path.join(savepathfol, str(j)+"registeredimg" + "." + "png")
        lilimage=image[pt[1]-w:pt[1]+w, pt[0]-w:pt[0]+w]
        performsaveimage(lilimage,savepath)
        
def blockoutunwantedparticles(analpath,sat_img,path,threshold,clumpflag):
## a function which blocks out that particles which are interfering with analysis i.e. clumps and clusters
#inputs: analpath: the path to the analysis folder
######## sat_img: the before target image in rgb format
#output: correctedimgcore, the corrected image after accounting for cores and clumps
# These images are hardcoded into a folder, so the code has some dependencies. But any exmaples of the correct size will do
    #The threshold of 0.25 seems to be highly variable
    correctedimg,_,r= performtemplatematching(sat_img, path,1,threshold,clumpflag)
    #plt.imshow(correctedimg)
    #print(r)
    return correctedimg,r


def unwantedparticleblocking(analpath,sat_img):
    #clusterpath=  r"D:\OneDrive - UNSW\Image_Analysis\typesofParticles\NewAItraining\Cluster"
    #threshold= 0.25
    #correctedimgcluster=blockoutunwantedparticles(analpath,sat_img,clusterpath,threshold)
    
    clumppath= r"D:\OneDrive - UNSW\Image_Analysis\typesofParticles\NewAItraining\newmicroscope\clump"
    threshold=0.55
    savepath=os.path.join(analpath, "clumpcorrectedimg" + "." + "png")
    correctedimgclump,r=blockoutunwantedparticles(analpath,sat_img,clumppath,threshold,0)
    performsaveimage(correctedimgclump,savepath)
    
    corepath= r"D:\OneDrive - UNSW\Image_Analysis\typesofParticles\NewAItraining\newmicroscope\core2"
    threshold=0.55
    correctedimgcore,r=blockoutunwantedparticles(analpath,correctedimgclump,corepath,threshold,1)
    savepath=os.path.join(analpath, "corecorrectedimg" + "." + "png")
    performsaveimage(correctedimgcore,savepath)
    
    return correctedimgcore,r
    
def keepmatchedparticles(path,image,threshold):
## a function which keeps the wanted particles and blocks out the rest. 
#inputs: analpath: the path to the analysis folder
######## sat_img: the before target image in rgb format
#########path: path to the images to analyse
    correctedimgdimer,pointsdimer,r= performtemplatematching(image, path, 0, threshold,1)
    
    return correctedimgdimer,pointsdimer,r
                                                              

def performkeepmatchedparticles(analpath,image):
## a function which keeps the wanted particles and blocks out the rest. 
#inputs: analpath: the path to the analysis folder
######## sat_img: the before target image in rgb format
#########path: path to the images to analyse
#outputs: the selected image and the dimer points selected
    dimerpath= r"D:\OneDrive - UNSW\Image_Analysis\typesofParticles\NewAItraining\newmicroscope\dimer2"
    threshold=0.55
    
    correctedimgdimer,pointsdimer,r=keepmatchedparticles(dimerpath,image,threshold)
    savepath=os.path.join(analpath, "pickeddimersimg" + "." + "png")
    performsaveimage(correctedimgdimer,savepath)
    return correctedimgdimer,pointsdimer,r



def getaveragevalueshsv(image,points,w,h,analpath):
# a function to get the average values of the image in rgb format
#inputs: image: the hsv image
#########points: the selected dimer values
#########w,h- the width and height in pixels
#########analpath: the path to the analysis folder
#outputs: an array of the average colour of the red divided by the green vector

    average_colour_rDivg=[]
    #converts the image to hsv
    image = cv.cvtColor(image,  cv.COLOR_RGB2HSV)
    image_hue=image[:,:,0]
    for i,pt in enumerate(points):           
        try: 
            crop_im=image_hue[pt[1]-w:pt[1]+w, pt[0]-w:pt[0]+w]
            dividedval=crop_im.reshape((1,-1))
            #print(dividedval)
            average_colour_rDivg.append(np.mean(dividedval))
            
        except:
            z=1;
    
    print("max is ",np.max(np.array(average_colour_rDivg)), " min is ",np.min(np.array(average_colour_rDivg)))
    print("average is ", np.mean(np.array(average_colour_rDivg)), "median is ", np.mean(np.array(average_colour_rDivg)) )
    print("standard deviation is ", np.mean(np.array(average_colour_rDivg)))
    return np.array(average_colour_rDivg)

def getaveragevaluesLAB(image,points,analpath):
    w=8
    labim = rgb2lab(image)
    l_vec,a_vec,b = cv.split(labim)
    average_colour_aDivg=[]
    max_val_lDivg=[]
    average_lum=[]
    #converts the image to hsv
    #image = cv.cvtColor(image,  cv.COLOR_RGB2HSV)
    #image_hue=np.true_divide(image[:,:,0], image[:,:,1], where=(image[:,:,0]!=0) | (image[:,:,1]!=0))
    #image_hue=image[:,:,0]
    for i,pt in enumerate(points):           
        try: 
            crop_im=a_vec[pt[1]-w:pt[1]+w, pt[0]-w:pt[0]+w]
            dividedval=crop_im.reshape((1,-1))
            
            crop_im_lval=l_vec[pt[1]-w:pt[1]+w, pt[0]-w:pt[0]+w]
            dividedval_lval=crop_im_lval.reshape((1,-1))
            #print(dividedval)
            average_colour_aDivg.append(np.mean(dividedval))
            
            max_val_lDivg.append(np.max(dividedval_lval))
            average_lum.append(np.mean(dividedval_lval))
            
        except:
            z=1;
    if not average_colour_aDivg:
        average_colour_aDivg=[0]
    
    print("max is ",np.max(np.array(average_colour_aDivg)), " min is ",np.min(np.array(average_colour_aDivg)))
    print("average is ", np.mean(np.array(average_colour_aDivg)), "median is ", np.median(np.array(average_colour_aDivg)) )
    print("standard deviation is ", np.std(np.array(average_colour_aDivg)))
    return np.array(average_colour_aDivg),np.array(average_lum),np.array(max_val_lDivg)

def averagehistogramshift(correctedimgdimer,pointsdimer,correctedimgtarget,threshold,analpath):
# a function which uses the average shift of the particle colour (either hue or rgb depending ) to 
#select the particles which have shifted in the after image 
#inputs: correctedimgdimer: the image with only dimers selected
#########pointsdimer: the locations of the dimers in the image
#########correctedimgtarget: the registered image after target added wherein the coordinates of the selected
# dimers have been used to clip it
#########threshold: the values which the shifted average value must be above or below. was 0.2 
#output: loc- the shifted selected particles. 
    avdim,ldim,maxldim=getaveragevaluesLAB(correctedimgdimer,pointsdimer,analpath)
    
    avcore,lcore,maxlcore=getaveragevaluesLAB(correctedimgtarget,pointsdimer,analpath)
    minus= np.array(avdim)-np.array(avcore)
    minus_l=np.array(ldim)-np.array(lcore)
    
    print("minus values here ------------")
    print("max value is ", np.max(minus), "min value is ", np.min(minus), " mean value is ", np.mean(minus))
    print(" median value is ", np.median(minus), " standard deviation is ", np.std(minus))
    loc=np.array(pointsdimer)[np.logical_and(minus>5,maxlcore>1)]
    ldim2=ldim[np.logical_and(minus>5,maxlcore>1)]
    loc=loc[ldim2<30]
    r=8
    #loc=np.array(pointsdimer)[np.logical_and(minus<threshold, minus > -3)]
    targetpicked= blockimagecombo(correctedimgtarget,loc,r,0)
    #average_values=np.array(minus)[np.logical_and(minus<threshold, minus > -3)]
    average_values=np.array(minus)[np.logical_and(minus>5,maxlcore>1)]
    #print(minus)
    #average_values=average_values[ldim2<30]
    #print(ldim2)
    
    savepath=os.path.join(analpath, "tarmaskedimg" + "." + "png")
    performsaveimage(targetpicked,savepath)
    return loc,average_values.tolist()
    



        
        
def performtemplatetrainingonimages(transtarimg,analpath,sat_img):
    

    #blocks unwated particles
    correctedimgcore,r=unwantedparticleblocking(analpath,sat_img)
    
    
    #selects dimers
    correctedimgdimer,pointsdimer,r=performkeepmatchedparticles(analpath, correctedimgcore)
    image_circle=blockimagecombo(sat_img, pointsdimer,16,2)
    savepath=os.path.join(analpath, "dimerCircled" + "." + "png")
    performsaveimage(image_circle,savepath)
    
    
    #print(np.unique(np.array(pointsdimer),axis=0))
    #pointsdimer_corrected=np.unique(np.array(pointsdimer),axis=0)
    pointsdimer_corrected=pointsdimer
    #uses those points to select the same points in the target image 
    
    transtarimg=makeclippingmask(transtarimg)
    correctedimgtarget=blockimagecombo(transtarimg,pointsdimer_corrected,r,0)
    
    savepath=os.path.join(analpath, "targetblocked" + "." + "png")
    performsaveimage(correctedimgtarget,savepath)
    
    #scans the before and after dimers to determine if the particles shift in hue
    threshold=-0.001
    selected_target_locations,average_values=averagehistogramshift(correctedimgdimer,pointsdimer_corrected,correctedimgtarget,threshold,analpath)
    image_circle=blockimagecombo(transtarimg, selected_target_locations,16,2)
    savepath=os.path.join(analpath, "targetCircled" + "." + "png")
    performsaveimage(image_circle,savepath)
    
    
    #comment out if don't want
    #savetotrainingfol(r"D:\OneDrive - UNSW\Image_Analysis\typesofParticles\NewAItraining\newmicroscope\unsorted\before",sat_img,pointsdimer_corrected)
    #savetotrainingfol(r"D:\OneDrive - UNSW\Image_Analysis\typesofParticles\NewAItraining\newmicroscope\unsorted\after",transtarimg,selected_target_locations)
    
    return selected_target_locations,pointsdimer_corrected,average_values


######### Extra functions

def getindividualfoldersandsuch(bigfol):
# a function which gets the folders underneath the big folder. Must be the format: bigfol>slide>sample>satellite | target> images
    topfolders=listdirs(bigfol)
    subfolders=[0]
    for nextfolders in topfolders:
        subfolders= subfolders+ listdirs(nextfolders)
    subfolders.pop(0)
    #print(subfolders)
    subfolders=np.array(subfolders).reshape(-1,1)
    #print(subfolders)
    return subfolders
def searchforsatandtargetfolders(folder):
#finds the satellite and targetfolders using regexp
    subfolders2 = folder.tolist()
    subfolders3=str(subfolders2).replace('[','').replace(']','').replace('\\\\','\\')

    listexpfolders=listdirs(subfolders3[1:len(subfolders3)-1])
    beforefol = [x for x in listexpfolders if re.search("satellite",x)]
    beforefol=beforefol[0]
    afterfol= [x for x in listexpfolders if re.search("target",x)]
    afterfol=afterfol[0]
    
    return beforefol,afterfol

def createanalysisfolder(beforefol,string):
#creates an analysis folder in the address above where the satellite and target folders are located.
    oneuppath=os.path.dirname(beforefol)
    analysisfolderpath=oneuppath+"\\"+string
    try:
        os.mkdir(analysisfolderpath)
    except:
        print("folder already exists")
    print(analysisfolderpath)
    print("processing images")
    return analysisfolderpath,oneuppath

def createimagesubfolderforsaving(pathtomatch,analysisfolderpath):
# creates a folder with the name of the image in the analysis folder 
    pathtomatch=beforeimfile[j]
    matchingsearch=re.search("IMG_.*.CR2",pathtomatch)
    savefilespath=analysisfolderpath+"\\"+matchingsearch.group()+"\\"
    try: 
        os.mkdir(savefilespath)
    except:
        print("folder already exists")
    return savefilespath
def addstringwithtime(savefilespath):
# adds the current date and time so there's no saving over the top of different analysis
    now = datetime.now()
    dt_string = now.strftime("%d_%m_%Y_%H_%M_%S")
    savepath=savefilespath+dt_string
    return savepath
def makeanalysisfolder(savefilespath,string):
# makes a directory in the folder which matches the image name which says 'analysis'
    analpath=os.path.join(savefilespath,string)
                        #saves the registered image
    try: 
        os.mkdir(analpath)
    except:
        print("folder already exists")
    
    return analpath

################ THRESHOLDING
def gethuevaluesthreshHSV(image,points):
    image_hsv = cv.cvtColor(image, cv.COLOR_RGB2HSV)
    image_hue=image_hsv[:,:,0]
    w=8
    average_colour_rDivg=[]
    for i,pt in enumerate(points):           
        try: 
            crop_im=image_hue[pt[1]-w:pt[1]+w, pt[0]-w:pt[0]+w]
            dividedval=crop_im.reshape((1,-1))
            #print(dividedval)
            average_colour_rDivg.append(np.mean(dividedval))
            
        except:
            z=1;
    return np.array(average_colour_rDivg)

def getcentroidsandcenters(image):
    
    #wmakes an excellent binary image
    src = image.copy()

    
    # Create a kernel that we will use to sharpen our image
    # an approximation of second derivative, a quite strong kernel
    kernel = np.array([[1, 1, 1], [1, -8, 1], [1, 1, 1]], dtype=np.float32)
    # do the laplacian filtering as it is
    # well, we need to convert everything in something more deeper then CV_8U
    # because the kernel has some negative values,
    # and we can expect in general to have a Laplacian image with negative values
    # BUT a 8bits unsigned int (the one we are working with) can contain values from 0 to 255
    # so the possible negative number will be truncated
    imgLaplacian = cv.filter2D(src, cv.CV_32F, kernel)
    sharp = np.float32(src)
    imgResult = sharp - imgLaplacian
    # convert back to 8bits gray scale
    imgResult = np.clip(imgResult, 0, 255)
    imgResult = imgResult.astype('uint8')
    
    # Create binary image from source image
    # Create binary image from source image
    bw = cv.cvtColor(imgResult, cv.COLOR_RGB2GRAY)
    #_, bw = cv.threshold(bw, 40, 255, cv.THRESH_BINARY | cv.THRESH_OTSU)
    contours, hierarchy = cv.findContours(bw, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
    list_of_centres=[]
    boundRect = [None]*len(contours)
    contours_poly = [None]*len(contours)
    for i, c in enumerate(contours):
        contours_poly[i] = cv.approxPolyDP(c, 3, True)
        center_circle, _ = cv.minEnclosingCircle(contours_poly[i])
        list_of_centres.append((np.int(np.round(center_circle[0])),np.int(np.round(center_circle[1]))))
        
        boundRect[i] = cv.boundingRect(contours_poly[i])
        cv.circle(src, ((np.int(np.round(center_circle[0])),np.int(np.round(center_circle[1])))), 20, (255, 255, 0), 1)
        
        
        #print((np.int(np.round(center_circle[0])),np.int(np.round(center_circle[1]))))
    #picked_rectangles=non_max_suppression_fast(np.array(boundRect),0.01)
    #h=16;
    #for (startX, startY, endX, endY) in picked_rectangles:
        #the coordinates are top left hand corner, bottom right hand corner for rectangles 
    #    list_of_centres.append((np.round((startX+endX/2)),np.round((startY+endY/2))))
        #cv.circle(src, (np.int(np.round((startX+endX/2))),np.int(np.round((startY+endY/2)))), 20, (255, 255, 0), 1)
        
    #cv.imshow("Image", src)
    #plt.show()
    #list_of_centres=np.array(list_of_centres)
    #print(list_of_centres)
    return list_of_centres


def performthresholding(beforeim,afterim,analpath):
    list_of_centres=getcentroidsandcenters(beforeim)
    average_colour=gethuevaluesthreshHSV(image,list_of_centres)
    dimer_points= np.array(list_of_centres)[average_colour <= 20]
    
    average_colour_dimer=gethuevaluesthreshHSV(image,dimer_points)
    dimers_blocked=blockimagecombo(beforeim, dimer_points,16,0)
    savepath=os.path.join(analpath, "dimersblocked" + "." + "png")
    performsaveimage(dimers_blocked,savepath)
    target_image=blockimagecombo(afterim, dimer_points,8,0)
    savepath=os.path.join(analpath, "targetblocked" + "." + "png")
    performsaveimage(target_image,savepath)
    average_colour_after=gethuevaluesthreshHSV(target_image,dimer_points)
    particle_difference=average_colour_dimer-average_colour_after
    target_points= np.array(dimer_points)[particle_difference < -10]
    target_image_picked=blockimagecombo(afterim, target_points,16,0)
    savepath=os.path.join(analpath, "targetpicked" + "." + "png")
    performsaveimage(target_image_picked,savepath)
    return dimer_points,target_points
    
#########################


subfolders=getindividualfoldersandsuch(bigfol)

for folder in subfolders:
    #automatically find the before and after folder 
    beforefol,afterfol= searchforsatandtargetfolders(folder)
    
    #Create the relevant folder for saving,as well as the path one folder up 
    analysisfolderpath_temp,oneuppath_temp=createanalysisfolder(beforefol,"Analysisfolder_Template")
    analysisfolderpath_thresh,oneuppath_thresh=createanalysisfolder(beforefol,"Analysisfolder_thresholding")
    
    
    #loadsall the images in 

    imaf,imbef,beforeimfile,afterimfile=imageprocessingfunction(beforefol,afterfol)
    print("processed and converted images")
    
    
    response=[0]*len(beforeimfile)
    dimercount=[0]*len(beforeimfile)
    targetcount=[0]*len(beforeimfile)
    response_thresh=[0]*len(beforeimfile)
    dimercount_thresh=[0]*len(beforeimfile)
    targetcount_thresh=[0]*len(beforeimfile)
    #t = tqdm(total=len(imbef))
    for j,image in enumerate(imbef):
        #with tqdm(total=len(imbef), file=sys.stdout) as pbar:

            #pbar.write('processed: %d' % (j+1))
            #pbar.update(0.25)
            #sleep(0.1)

        pathtomatch=beforeimfile[j]
        savefilespath_temp= createimagesubfolderforsaving(pathtomatch,analysisfolderpath_temp)
        savefilespath_thresh= createimagesubfolderforsaving(pathtomatch,analysisfolderpath_thresh)
        
        
        #add on the currentdate and time 
        savepath_temp=addstringwithtime(savefilespath_temp)
        savepath_thresh=addstringwithtime(savefilespath_thresh)
        
        
        print("calculating image "+str(j+1)+" of "+ str(len(imbef)))
        
        #make an analysis folder which says analysis in the image folder just created 
        analpath_temp=makeanalysisfolder(savefilespath_temp,"Analysisimages_Template")
        analpath_thresh=makeanalysisfolder(savefilespath_thresh,"Analysisimages_thresholding")
        
        #spot matches the before and after images 
        transimaf=imgregfun(image, imaf[j])
        #saves the redistered image
        saveregisteredimage=os.path.join(analpath_temp, "registeredimg" + "." + "png")
        performsaveimage(transimaf,saveregisteredimage)
        #with tqdm(total=len(imbef), file=sys.stdout) as pbar:

            #pbar.write('processed: %d' % (j+1))
        #    pbar.update(0.5)
        #    sleep(0.1)
        
        #uncomment if you want a crop box    
        #height, width = image.shape[:2]
        #boxwid=round(1500/2)
        #centreimagebefore=image[round(height/2)-boxwid:round(height/2)+boxwid,round(width/2)-boxwid:round(width/2)+boxwid]
        #centreimageafter=transimaf[round(height/2)-boxwid:round(height/2)+boxwid,round(width/2)-boxwid:round(width/2)+boxwid]
        centreimagebefore=makeclippingmask(image)
        print("Made the clipping Mask")
        #plt.imshow(centreimagebefore)
        centreimageafter=makeclippingmask(transimaf)
        
        #with tqdm(total=len(imbef), file=sys.stdout) as pbar:

            #pbar.write('processed: %d' % (j+1))
            #pbar.update(0.75)
            #sleep(0.1)
        dimer_points,target_points=performthresholding(centreimagebefore,centreimageafter,analpath_thresh)
        
        #perform the template matching and see which particles shift
        targetpicked,pointsdimer,average=performtemplatetrainingonimages(centreimageafter,analpath_temp,centreimagebefore)
       
        
        #saves the to an excel spreadsheet and saves the results to a list to get the overall results at the end 
        #if I feel bothered I can add something in that uses the already analysed images but meh 
        dimercount[j]=len(pointsdimer)
        targetcount[j]=len(targetpicked)
        try: 
            response[j]=(len(targetpicked)/len(pointsdimer))*100
        except:
            response[j]=0
        
       # saveexcelfun(pointsdimer,targetpicked,savepath_temp)
        
        dimercount_thresh[j]=len(dimer_points)
        targetcount_thresh[j]=len(target_points)
        try: 
            response_thresh[j]=(len(target_points)/len(dimer_points))*100
        except:
            response_thresh[j]=0
        #print(pointsdimer.type)
        #print(dimer_points.type)
        #saveexcelfun(dimer_points,target_points,savepath_thresh)
  
        
        
        #save important coordinates to a text file for later analysis
        #Probably can delete this don't know what it's used for anymore 
        #savetextfilefun(np.array(pointsdimer),savepath,"dimerbefore")
        #savetextfilefun(np.array(targetpicked),savepath,"selectedResponse")

    print("Finished analysis woohoo")
    #t.close()
    #Find the folders which contain the analysis and read the text files
    #Find the image folders
    searchfolder=oneuppath_temp
    #insearchfolder=os.listdir(searchfolder)
    savedir_temp= searchfolder+'\\'+"Analysisfolder_Template"
    imagefilefolders=os.listdir(savedir_temp)
    #Write excel worsheet in the analysis folder
    responseforexcel = pd.DataFrame({'dimers_picked':np.array(dimercount),'target_picked':np.array(targetcount),'response': np.array(response)})
    responsepath_temp=savedir_temp+'\\'+"responseforallimages.xlsx"
    responseforexcel.to_excel(responsepath_temp) 
    
    searchfolder_thresh=oneuppath_thresh
    #insearchfolder=os.listdir(searchfolder)
    savedir_thresh= searchfolder+'\\'+"Analysisfolder_thresholding"
    imagefilefolders=os.listdir(savedir_thresh)
    #Write excel worsheet in the analysis folder
    responseforexcel = pd.DataFrame({'dimers_picked':np.array(dimercount_thresh),'target_picked':np.array(targetcount_thresh),'response': np.array(response_thresh)})
    responsepath_thresh=savedir_thresh+'\\'+"responseforallimages.xlsx"
    responseforexcel.to_excel(responsepath_thresh) 


C:\Users\Image_Processing_PC\OneDrive\OneDrive - UNSW\Image_Analysis\NewMicroscope\Time_series\slide1\Time series in order\Analysisfolder_Template
processing images
C:\Users\Image_Processing_PC\OneDrive\OneDrive - UNSW\Image_Analysis\NewMicroscope\Time_series\slide1\Time series in order\Analysisfolder_thresholding
processing images
processed and converted images
calculating image 1 of 7
Made the clipping Mask


c:\program files\python36\lib\site-packages\numpy\core\fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
c:\program files\python36\lib\site-packages\numpy\core\_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

...

max is  25.922435229848293  min is  -4.588185583844274
average is  5.49518528694137 median is  4.7230651259785486
standard deviation is  4.0692600087068245
max is  23.102424014334673  min is  -13.592569929469477
average is  3.6891047829219152 median is  3.3515122271625115
standard deviation is  4.149480841357585
minus values here ------------
max value is  21.22882681427975 min value is  -10.89530842604268  mean value is  1.8060805040194552
 median value is  1.4569536216661123  standard deviation is  2.590474443817208
calculating image 7 of 7
Made the clipping Mask


  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

max is  25.922435229848293  min is  -4.588185583844274
average is  5.49518528694137 median is  4.7230651259785486
standard deviation is  4.0692600087068245
max is  26.189542031142977  min is  -13.872780240257633
average is  3.8794510735696335 median is  3.500092190109373
standard deviation is  4.216061604118147
minus values here ------------
max value is  18.42635941608959 min value is  -11.201773648414  mean value is  1.615734213371737
 median value is  1.2978058602253182  standard deviation is  2.5934486564119292
Finished analysis woohoo


#################### TEMPLATE MATCHING MAIN
